In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

In [ ]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 946.8 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
nlp=spacy.load("en_core_web_lg")
nlp

In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
nlp=spacy.blank("en")
db=DocBin()

In [ ]:
import json
f=open("training_data.json")
train_data=json.load(f)

In [ ]:
train_data

{'classes': ['DATE', 'PERSON', 'COMPANY', 'MONEY', 'LOCATION'],
 'annotations': [['In 2022, Sundar Pichai announced that Google earned $280 billion in revenue from its offices in California.\r',
   {'entities': [[96, 106, 'LOCATION']]}],
  ['', {'entities': []}]]}

In [ ]:
for text, annot in tqdm(train_data['annotations']):
    doc = nlp.make_doc(text)  # Create a Doc object
    ents = []
    for start, end, label in annot['entities']:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents
    db.add(doc)

100%|██████████| 2/2 [00:00<00:00, 695.40it/s]


In [ ]:
db.to_disk("./training_data.spacy")

In [ ]:
!python -m spacy init config config.cfg --lang en --pipeline ner --optimize accuracy

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: accuracy
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

ℹ Saving to output directory: .
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     12.55    0.00    0.00    0.00    0.00
200     200          2.57    199.89  100.00  100.00  100.00    1.00
400     400          0.00      0.00  100.00  100.00  100.00    1.00
600     600          0.00      0.00  100.00  100.00  100.00    1.00
800     800          0.00      0.00  100.00  100.00  100.00    1.00
1000    1000          0.00      0.00  100.00  100.00  100.00    1.00
1200    1200          0.00      0.00  100.00  100.00  100.00    1.00
1400    1400          0.00      0.00  100.00  100.00 

In [ ]:
nlp_ner=spacy.load("/content/model-best")

In [ ]:
doc = nlp_ner("""In 2024, Sundar Pichai announced that Google earned $320 billion from its headquarters in California.
Elon Musk revealed in 2023 that Tesla invested $5 billion in Texas to expand its factories.
Satya Nadella confirmed in 2022 that Microsoft generated $250 billion from offices in Washington.
Jeff Bezos donated $200 million to Amazon projects in India in 2021.
Mark Zuckerberg shared in 2020 that Meta invested $12 billion in London.
Narendra Modi declared in 2023 that Infosys achieved revenue of $50 million in Bengaluru.
Tim Cook reported in 2021 that Apple reached $300 billion in global revenue based in Cupertino.
Bill Gates stated in 2019 that Microsoft contributed $20 billion to healthcare programs in New York.
Elon Musk promised in 2025 that SpaceX secured $2 billion in funding from investors in Florida.
Sundar Pichai mentioned in 2022 that Google opened new data centers worth $1 billion in Tokyo.
""")

In [ ]:
spacy.displacy.render(doc,style="ent")